In [6]:
from ddpm import *
from unet import Unet
import wandb
import config as cfg

import os
from pathlib import Path
from torchvision.utils import save_image


from utils import *
from preprocess import *

def binarize(img):
    b_img = img.astype(np.float32)
    b_img = 255 * (b_img - np.min(b_img)) / (np.max(b_img) - np.min(b_img))
    b_img = b_img.astype(np.uint8)

    blured = cv2.GaussianBlur(b_img, create_kernel(b_img, 50), 0)

    otsu_tr = threshold_otsu(blured) * 0.8
    mask = np.where(blured >= otsu_tr, 1, 0).astype(np.uint8)

    return mask

In [9]:
inf_num = 7
os.environ['CUDA_VISIBLE_DEVICES'] = f'{inf_num}'
device = "cuda" if torch.cuda.is_available() else "cpu"
# torch.cuda.set_device(cfg.inference_num)

patches_artifact_dir = '/home/nikola.jovisic.ivi/exp_mammo/rsna/pipeline/artifacts/rsna_3c_256:v70'

model_patch = Unet(dim=128, dim_mults=(1, 2, 2, 4, 4), channels=3)
model_checkpoint = torch.load(f'{patches_artifact_dir}/model_131999.pt', map_location='cpu')
model_patch.load_state_dict((dict([(n.replace('module.', ''), p) for n, p in model_checkpoint['model_state'].items()])))

# samples_folder = '/lustre

<All keys matched successfully>

In [13]:
samples_folder = '/lustre/3c-anomaly'
if not os.path.exists(samples_folder):
    os.mkdir(samples_folder)
        
for i in range(14898):
    if i % 8 == inf_num:
        print(i)
        with torch.no_grad():
            save_dir = os.path.join(samples_folder, f'img_{str(i)}.png')
            
            if not os.path.exists(lc_dir):
                os.makedirs(lc_dir)
            if not os.path.exists(patches_dir):
                os.makedirs(patches_dir)
            
            # load whole image
            img = cv2.imread(f'/lustre/samples/rsna/v1/whole_images_original/img_{i}.png')
            img = img[:, :, 0]/255
            print(img.shape)
            break

7
(256, 256)


In [14]:
            transform = Compose([ToTensor(), Normalize(img.mean(), img.std())])
            img = transform(img)
            
            # generate local contexts
            img_channels = create_channels_lcl_ctx(img)
            inputs, black_idx = create_inputs(img_channels, img)
            batch = torch.Tensor(np.array(inputs))
            local_contexts = generate_patches(local_context_model, batch, device)
            
            lcl_img = np.zeros((NUM_PATCHES*256, NUM_PATCHES*256))

            i = 0
            j = 0
            n = 0
            num_rows = NUM_PATCHES
            min_value = local_contexts.cpu().min()
            max_value = local_contexts.cpu().max()
            for idx in range(len(img_channels)):
                if idx in black_idx:
                    patch = torch.zeros((256, 256)) - abs(min_value)
                else:
                    patch = local_contexts[n]
            #         patch = (patch - patch.min()) / (patch.max() - patch.min())
                    patch = patch[0].cpu()
                    n += 1
            #     if i % 3 == 1 and j % 3 == 1:
                lcl_img[j*256:(j+1)*256, i*256:(i+1)*256] = patch

                i += 1
                if i == num_rows:
                    j += 1
                    i = 0
            
            # generate patches
            patch_inputs, black_idx_patches = create_patch_inputs_from_lcl_ctx_img(lcl_img, img)
            
            n = 0
            patches_arr = []
            while n < len(patch_inputs):
                patch_batch = torch.Tensor(np.array(patch_inputs[n:n+40]))
                patches = generate_patches(model_patch, patch_batch, device)
                patches_arr.append(patches)
                n += 40
                
            patches = torch.cat(patches_arr, dim=0)

            # final image
            lc_normalized = (local_contexts - local_contexts.min()) / (local_contexts.max() - local_contexts.min())
            patches_normalized = (patches - patches.min()) / (patches.max() - patches.min())
            
            
            num_patches = int(np.round(ORIG_IMG_REAL_SIZE/PATCH_REAL_SIZE))

            final = np.zeros((num_patches*256, num_patches*256))
            i = 0
            j = 0
            n = 0
            num_rows = num_patches
            for idx in range(num_patches*num_patches):
                if idx in black_idx_patches:
                    patch = torch.zeros((256, 256))
                else:
                    patch = patches_normalized[n]
            #         patch = (patch - patch.min()) / (patch.max() - patch.min())
                    patch = patch[0].cpu()
                    n += 1
            #     if i % 3 == 1 and j % 3 == 1:
                final[j*256:(j+1)*256, i*256:(i+1)*256] = patch
                i += 1
                if i == num_rows:
                    j += 1
                    i = 0    
                    
            # save images
            lcl_img = (lcl_img - lcl_img.min()) / (lcl_img.max() - lcl_img.min())

            save_image(torch.tensor(final), str(Path(save_dir) / f'pipeline.png'))
            save_image(torch.tensor(lcl_img), str(Path(save_dir) / f'local_contexts_img.png'))
            for i, lc in enumerate(lc_normalized):
                save_image(lc[0], str(Path(lc_dir) / f'local_context-{i}.png'))
            for i, patch in enumerate(patches_normalized):
                save_image(patch[0], str(Path(patches_dir) / f'patch-{i}.png'))
                
            mask = binarize(final)
            mask = dilate(mask)
            test = final*mask
            save_image(torch.tensor(test), str(Path(save_dir) / f'processed.png'))

            del local_contexts
            del patches
            torch.cuda.empty_cache() 

NameError: name 'local_context_model' is not defined